In [42]:
from itertools import combinations
import numpy as np
from z3 import *
from functools import reduce
import sympy as sp
from sympy import Mul, Pow, Add
x=sp.Symbol('x')
y=sp.Symbol('y')
def antimap(f):
    f = sp.expand(f)
    term=f.as_ordered_terms()
    exp=0
    for i in range(len(term)):
        coef,vari=term[i].as_coeff_Mul()
        exp=exp+coef%2*vari**-1
    return exp
def commute(a,b):
    a = sp.Matrix(a).T
    b = sp.Matrix(b).T
    lamb=sp.Matrix([[0,0,1,0],
          [0,0,0,1],
          [-1,0,0,0],
          [0,-1,0,0]])
    aa=[]
    for i in range(len(a)):
        aa.append(antimap(a[i]))
    aa=sp.Matrix(aa)
    f=(aa.T*lamb*b)[0]
    f = sp.expand(f)
    term=f.as_ordered_terms()
    exp=0
    for i in range(len(term)):
        coef,vari=term[i].as_coeff_Mul()
        exp=exp+coef%2*vari
    return sp.simplify(exp)
x1=sp.Matrix([1,0,0,0]).T
x2=sp.Matrix([0,1,0,0]).T
z1=sp.Matrix([0,0,1,0]).T
z2=sp.Matrix([0,0,0,1]).T
basis=[]
basis.append(x1)
basis.append(x2)
basis.append(z1)
basis.append(z2)
def excimap(*A,oper):
    s=[]
    for i,M in enumerate(A):
        s.append(M)
    excitation=[]
    for i in range(len(s)):
        excitation.append(commute(s[i],oper))
    return sp.Matrix(excitation).T
def TDmap(f,m):
    vari=[]
    for j in range(2*m+1):
        for i in range(2*m+1):
            vari.append(x**(i-m)*y**(j-m))
    s=[]
    for i in range(len(vari)):
        s.append(vari[i]*f)
    return sp.Matrix(s).T

In [44]:
s1=(1-1/x)*x1+(1-1/y)*x2
s2=(1-y)*z1+(x-1)*z2
print(excimap(s1,s2,oper=x1))
print(excimap(s1,s2,oper=x2))
print(excimap(s1,s2,oper=z1))
print(excimap(s1,s2,oper=z2))


Matrix([[0, (y + 1)/y]])
Matrix([[0, (x + 1)/x]])
Matrix([[x + 1, 0]])
Matrix([[y + 1, 0]])


[[(y + 1)/y, 0, 0, 0], [(x + 1)/x, 0, 0, 0], [x + 1, 0, 0, 0], [y + 1, 0, 0, 0]]
